In [2]:
import numpy as np
np.set_printoptions(precision=5, suppress = True)

import models.mirror_descent as mirror_descent
import structures.objectives as objectives
import models.restarter as restarter
import structures.oracles as oracles
import sklearn.linear_model as lm
import draw.main as draw
import exp_setup as es
from other.utils import *
import pickle

def rell():
    for module in [objectives, oracles, draw, mirror_descent, restarter, es]:
        imp.reload(module)
rell()
def warn(*args, **kwargs): pass
import warnings; warnings.warn = warn
opj = os.path.join

In [2]:
warnings.filterwarnings("ignore") 

### Generation for comparison with SMD

In [ ]:
import matplotlib.patches as mpatches
import matplotlib
    
p = {'L': 1.0, 'dim': 100000, 's': 50}
do_median, spacing = True, 4*50
settings = [(1.0,0.001), (1.0, 0.1), (0.1, 0.001), (0.1, 0.1)]
noise_types = ['normal', 't', 'hadamard']

d, p['noise'] = p['dim'], 0

betas = [0.1, 0.01, 0.5]
b  = betas[p['noise']]
if p['noise'] == 2: d = 65536

nst = np.sqrt
legend_titles = ['SMD-SR', 'SMD']
for i in range(4):
    fig = plt.figure(figsize=(17, 8.0))
    ax  = fig.add_subplot(111)
    p['mu'], p['sigma'] = settings[i]
    d_m_0, d_q_025_0, d_q_075_0 = get_aggregated_results(p,0, abs(b),d, med=1)
    d_m_1, d_q_025_1, d_q_075_1 = get_aggregated_results(p,2, abs(b),d, med=1)
    
    d_m_0, d_q_025_0, d_q_075_0 = nst(d_m_0), nst(d_q_025_0), nst(d_q_075_0)
    d_m_1, d_q_025_1, d_q_075_1 = nst(d_m_1), nst(d_q_025_1), nst(d_q_075_1)
    
    pack = [d_m_0, d_m_1]
    pack_025, pack_075 = [d_q_025_0, d_q_025_1], [d_q_075_0, d_q_075_1]
    if p['noise'] == 1:
        for j in range(3):
            pass #pack_025[j] = pack[j]; pack_075[j] = pack[j]
    limit = 3000
    pack=common_shorten(pack, limit=limit); 
    pack_025=common_shorten(pack_025, limit=limit); 
    pack_075=common_shorten(pack_075, limit=limit)

    codes = [2, 3, 1]
    colors = [line_colors[c] for c in codes]; colors_var = [face_colors[c] for c in codes]    
    
    SMALL_SIZE = 8; MEDIUM_SIZE = 10; BIGGER_SIZE = 12
    inds = np.arange(len(pack))
    styles = ["solid", "dashed"]
    for ind, line, c, q_025, q_075, var_c in zip(inds, pack, colors, pack_025, pack_075, colors_var):
        x = [i for i in np.arange(len(line)) * spacing if i <= 5*10**4]
        plt.plot(x, line[:len(x)], lw=1.5, c=c, ls=styles[ind])
        plt.fill_between(x, q_025[:len(x)], q_075[:len(x)], alpha=0.2, edgecolor='#CC4F1B', 
                                         facecolor=var_c, lw=0)
        plt.yscale( "log" );      plt.grid(1)

        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    
        xpos = 0.3 if p['sigma']==0.001 else 0.65
        plt.text(0.7, 0.95, r"$\kappa_\Sigma={}, \sigma={}$".format(*settings[i]), 
                             fontsize=26, transform=ax.transAxes,
                             verticalalignment='top', bbox=props)
        if i == 0:
            plt.legend(loc=3, 
                  handles=[mpatches.Patch(color=c, label=lt) 
                           for c, lt in zip(colors, legend_titles)])
    plt.rcParams['grid.alpha'] = 0.4
    root = **YOUR ROOT TO SAVE FILES**
    name = "_".join(my_map(str, [p['dim'], p['mu'], p['sigma'], p['s'], p['noise']]))
    print(name)
    # plt.savefig(opj(root, name + '.eps'), format='eps')
    plt.show()



### Generation for Lasso

In [11]:
face_colors = { 1: 'mistyrose', 2: 'lightblue', 3: 'beige', 5: 'peachpuff', 6: 'darkblue', 7: 'mistyrose', 
               8: 'lightsteelblue', 9: "beige", 11: "peachpuff", 12: 'lightsteelblue', 17: "lightpink"}
face_colors = { 1: 'lightcoral', 2: 'lightblue', 3: 'palegreen', 5: 'peachpuff', 6: 'darkblue', 7: 'lightcoral', 
               8: 'lightsteelblue', 9: "palegreen", 11: "peachpuff", 12: 'lightsteelblue', 17: "lightpink", 
               18: "palegreen", 19: "peachpuff"}

In [ ]:
import matplotlib.patches as mpatches
import matplotlib
    
p = {'L': 1.0, 'dim': 50000, 's': 50}

noise_types = ['normal', 't', 'hadamard'] # p['noise'] is the index of the noise model
d, p['noise'] = p['dim'], 1
do_median, spacing = True, 200
betas = [0.2, 0.005, 2.5]
beta  = betas[p['noise']]
sigma = 0.1
settings_list = [[(1.0, sigma), (0.1, sigma)], [(1.0, sigma), (0.1, sigma)]]
# settings_list = [[(1.0, 0.2), (0.1, 0.2)], [(1.0, 0.2), (0.1, 0.2)]]
settings = settings_list[p['noise']]
if p['noise'] == 2: d = 32768*2

legend_titles = ['SMD-SR', 'Lasso']
nst = np.sqrt
for i in range(2):
    fig = plt.figure(figsize=(17, 8.0))
    ax  = fig.add_subplot(111)
    p['mu'], p['sigma'] = settings[i]
    print(p['sigma'])
    d_m_0, d_q_025_0, d_q_075_0 = get_aggregated_results(p, 0, abs(beta), d, med=1)
    d_m_1, d_q_025_1, d_q_075_1 = get_aggregated_results(p, 7, "", d, med=1)
    
    d_m_0, d_q_025_0, d_q_075_0 = nst(d_m_0), nst(d_q_025_0), nst(d_q_075_0)
    d_m_1, d_q_025_1, d_q_075_1 = nst(d_m_1), nst(d_q_025_1), nst(d_q_075_1)
    
    # d_m_1, d_q_025_1, d_q_075_1 = d_m_1[::50], d_q_025_1[::50], d_q_075_1[::50]
    
    pack = [d_m_0, d_m_1]
    pack_025, pack_075 = [d_q_025_0, d_q_025_1], [d_q_075_0, d_q_075_1]
    if p['noise'] == 1:
        for j in range(2):
            pass # pack_025[j] = pack[j]; pack_075[j] = pack[j]
            
    limit = 4000
    pack=common_shorten(pack, limit=limit); 
    pack_025=common_shorten(pack_025, limit=limit); 
    pack_075=common_shorten(pack_075, limit=limit)
    
    codes = [2, 3, 1]
    colors = [line_colors[c] for c in codes]; colors_var = [face_colors[c] for c in codes]    
    
    SMALL_SIZE = 8; MEDIUM_SIZE = 10; BIGGER_SIZE = 12
    inds = np.arange(len(pack))
    
    styles = ["solid", "dashed"]
    for ind, line, c, q_025, q_075, var_c in zip(inds, pack, colors, pack_025, pack_075, 
                                                 colors_var):
        x = np.arange(len(line)) * spacing
        plt.plot(x, line, lw=1.3, c=c, ls=styles[ind])
        if ind==1:
            print(x[:5], line[:5], q_025[:5], q_075[:5])
        plt.fill_between(x, q_025, q_075, alpha=0.2, edgecolor='#CC4F1B', 
                                         facecolor=var_c, lw=0)
        plt.yscale( "log" )
        plt.grid(1)

        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        
        settings[i] = tuple([settings[i][0], sigma])
        xpos = 0.3 if p['sigma']==0.001 else 0.65
        plt.text(0.7, 0.95, r"$\kappa_\Sigma={}, \sigma={}$".format(*settings[i]), 
                            fontsize=26, transform=ax.transAxes,
                            verticalalignment='top', bbox=props)
    if i == -1:
        plt.legend(loc=3, handles=[mpatches.Patch(color=c, label=lt) 
                                   for c, lt in zip(colors, legend_titles)])
    plt.rcParams['grid.alpha'] = 0.4
    
    root = **YOUR ROOT TO SAVE FILES**
    name = "_".join(my_map(str, [p['dim'], p['mu'], p['sigma'], p['s'], p['noise']]))
    print(name)
    plt.savefig(opj(root, name + '.eps'), format='eps')
    plt.show()